In [1]:
from sklearn.datasets import load_boston 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np 

In [16]:
# Coletar os dados
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS','AGE'], axis=1)
log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=['price'])

In [21]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

# property_stats = np.ndarray(shape=(1, 11))
# property_stats[0][CRIME_IDX] = features['CRIM'].mean()
# property_stats[0][ZN_IDX] = features['ZN'].mean()
# property_stats[0][CHAS_IDX] = features['CHAS'].mean()

property_stats = features.mean().values.reshape(1, 11)

In [22]:
regr = LinearRegression().fit(features, target)

In [23]:
fitted_vals = regr.predict(features)

In [26]:
MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

In [44]:
def get_log_estimate(nr_rooms, students_per_classroom, next_to_river=False, high_confidence=True):
    # Configurar propriedade
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
    # Executar previsão
    log_estimate = regr.predict(property_stats)[0][0]
    
    # Calcular o Range
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE   
        interval = 68
    return log_estimate, upper_bound, lower_bound, interval

In [45]:
get_log_estimate(3, 20, next_to_river=True)

(2.7767581914803987, 3.1517824618746593, 2.401733921086138, 95)

In [46]:
get_log_estimate(3, 20, next_to_river=True, high_confidence=False)

(2.7767581914803987, 2.964270326677529, 2.5892460562832684, 68)

In [47]:
np.median(boston_dataset.target)

21.2

In [55]:
ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE/np.median(boston_dataset.target)
log_est, upper, lower, conf = get_log_estimate(9, students_per_classroom=15, next_to_river=False, high_confidence=False)

# Converter para o dólar atual
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR

# Arredondar os valores em dólares
rounded_est = np.around(dollar_est, -3)
rounded_hi = np.around(dollar_hi, -3)
rounded_low = np.around(dollar_low, -3)

print(f'O valor estimado da propriedade é {rounded_est}')
print(f'Em {conf}% de confiança o valuation range é')
print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end')

O valor estimado da propriedade é 827000.0
Em 68% de confiança o valuation range é
USD 685000.0 at the lower end to USD 997000.0 at the high end


In [70]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    """
    Estimate the price of property in Boston
    
    Keyword Arguments:
    rm -- number of rooms in the property
    ptratio -- number of students per teacher in the classroom for the school in the area
    chas -- True if the property is next to the river, False otherwise
    large_range -- True for 95% prediction interval, False for 68% interval
    """
    if rm < 1 or ptratio < 1:
        print('That is unrealistic. Try Again')
        return
    
    log_est, upper, lower, conf = get_log_estimate(rm, students_per_classroom=ptratio, next_to_river=chas, high_confidence=large_range)

    # Converter para o dólar atual
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    # Arredondar os valores em dólares
    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'O valor estimado da propriedade é {rounded_est}')
    print(f'Em {conf}% de confiança o valuation range é')
    print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end')

In [71]:
get_dollar_estimate(rm=3, ptratio=40)

O valor estimado da propriedade é 188000.0
Em 95% de confiança o valuation range é
USD 129000.0 at the lower end to USD 274000.0 at the high end
